In [1]:
import pandas as pd

from wsdata.consts import ZHUZHAI, BIESHU, SHANGYE, BANGONG, QUANSHI
from wsdata.models import WinsunDatabase
from wsdata.utils import wan


class OneUsage:
    def __init__(self, usage):
        self.usage = usage
        self.ws = WinsunDatabase()
        
    def __gxj(self, window, by):
        """
        :param window: int
        :param by: 'trend', 'plate'
        """
        return self.ws.gxj(by, 'month', window, usage=self.usage)
    
    @property
    def trend_table(self):
        window = 25 if self.usage == ZHUZHAI else 13
        df = self.__gxj(window, 'trend').df_adjusted
        df.index = [x[2:] for x in df.index]
        return df
    
    @property
    def trend_argue(self):
        shuoli = self.__gxj(13, 'trend').shuoli(0, 12).full_text
        return shuoli
    
    @property
    def plate_table(self):
        window = 1
        df = self.__gxj(1, 'plate').df_adjusted
        df.fillna(0, inplace=True)
        return df
    
    def __top3(self, by):
        """
        :param by: 'sale', 'sold'
        """
        def top(groupby, plate=QUANSHI):
            plate = '仙西' if plate=='仙林' else plate
            df =  self.ws.rank(f'month_{by}', 1, groupby, '面积', usage=self.usage, plate=plate).head(3)
            df.面积 = wan(df.面积)
            return df
        
        # 右表
        plate_top = top('板块')
                
        # 左表
        frames = (top(['板块', 'popularizename'], plate) for plate in plate_top.板块)
        prj_top = pd.concat(frames)
                
        # join
        df = pd.merge(prj_top, plate_top, on='板块')
        df.板块 = df[['板块','面积_y']].apply(lambda x: f'{x.板块}（{x.面积_y}万㎡）', axis=1)
        df.set_index(['板块', 'popularizename'], inplace=True)
        df.drop(['rank_x', 'rank_y', '面积_y'], axis=1, inplace=True)
        
        return df
    
    @property
    def plate_sale_top(self):
        return self.__top3('sale')
    
    @property
    def plate_sold_top(self):
        return self.__top3('sold')
    
    def rank(self, by):
        """
        :param by: 'money', 'space'
        """
        outputs = ['面积','金额','均价'] if by=='space' else ['金额','面积','均价']
        head = 10 if self.usage==ZHUZHAI else 5
        df = self.ws.rank('month_sold', 1, ['popularizename','板块'], outputs, usage=self.usage).head(head)
        df.金额 = wan(df.金额)
        df.均价 = df.均价.astype(int)
        df.set_index('rank', inplace=True)
        return df

### 修改物业类型

In [2]:
ou = OneUsage(BIESHU)

d:\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['8']; using no arguments.
  (coltype, args))


### 量价走势

In [3]:
ou.trend_table

d:\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['8']; using no arguments.
  (coltype, args))


,sale,sold,price
1705,1.82,6.12,16165.0
1706,2.93,4.50,24660.0
1707,5.61,4.81,19150.0
1708,0.10,4.55,24056.0
1709,1.65,3.72,28317.0
1710,0.55,2.08,23556.0
1711,9.99,3.13,22423.0
1712,9.37,5.67,21656.0
1801,2.62,5.38,28060.0
1802,4.62,4.92,20031.0


In [4]:
ou.trend_argue

d:\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['8']; using no arguments.
  (coltype, args))


'上市3.51万㎡，环比增长153%，同比增长93%。成交4.18万㎡，环比下降24%，同比下降32%。成交均价28315元/㎡，环比增长24%，同比增长75%。'

### 板块量价

In [10]:
ou.plate_table

d:\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['8']; using no arguments.
  (coltype, args))


,sale,sold,price
板块,,,
城中,0.00,0.39,49943.0
城东,0.00,0.00,0.0
城南,0.00,0.00,0.0
河西,0.00,0.04,29288.0
城北,0.42,0.00,0.0
仙林,0.00,0.15,26806.0
江宁,2.26,1.86,29369.0
浦口,0.00,1.10,27714.0
江北新区直管区,0.00,0.09,24850.0


In [11]:
ou.plate_sale_top

面积_x
板块         popularizename      
江宁（2.26万㎡） 梁台煦府            1.22
           融创玖溪桃花源         1.03
高淳（0.83万㎡） 双湖壹号公馆          0.83
城北（0.42万㎡） 电建洺悦府(乐居雅花园二期)  0.42

In [12]:
ou.plate_sold_top

面积_x
板块         popularizename      
江宁（1.86万㎡） 瑞安翠湖山           0.47
           银亿东城            0.47
           九间堂             0.41
浦口（1.1万㎡）  金基九月森林          0.62
           正源尚峰尚水          0.23
           北江锦城            0.12
城中（0.39万㎡） 万科安品园舍          0.39

### 排行榜

In [13]:
ou.rank('space')

,popularizename,板块,面积,金额,均价
rank,,,,,
1,金基九月森林,浦口,6164.83,21013.45,34086
2,瑞安翠湖山,江宁,4711.96,9434.93,20023
3,银亿东城,江宁,4699.99,13355.81,28417
4,九间堂,江宁,4145.56,18397.61,44379
5,万科安品园舍,城中,3896.85,19462.00,49943


In [14]:
ou.rank('money')

,popularizename,板块,金额,面积,均价
rank,,,,,
1,金基九月森林,浦口,21013.45,6164.83,34086
2,万科安品园舍,城中,19462.00,3896.85,49943
3,九间堂,江宁,18397.61,4145.56,44379
4,银亿东城,江宁,13355.81,4699.99,28417
5,瑞安翠湖山,江宁,9434.93,4711.96,20023


### 说理辅助

In [20]:
ou.ws.rank('month_sale', 1, 'popularizename', '件数', usage=BIESHU)

,rank,popularizename,件数
0,1,融创玖溪桃花源,47
1,2,梁台煦府,38
2,3,双湖壹号公馆,28
3,4,电建洺悦府(乐居雅花园二期),16
